In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from tqdm import tqdm

from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split

import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPool2D
from keras.utils import to_categorical
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical

from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

import cv2
import os

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
labels = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']
img_size = 64
def get_data(data_dir):
    data = [] 
    for label in labels: 
        path = os.path.join(data_dir, label)
        class_num = labels.index(label)
        for img in tqdm(os.listdir(path)):
            try:
                img_arr = cv2.imread(os.path.join(path, img))[...,::-1] #convert BGR to RGB format
                resized_arr = cv2.resize(img_arr, (img_size, img_size)) # Reshaping images to preferred size
                data.append([resized_arr, class_num])
            except Exception as e:
                print(e)
    return np.array(data)

In [4]:
images_data = get_data("gdrive/My Drive/emotions_data")

100%|██████████| 4002/4002 [00:10<00:00, 397.92it/s]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  from ipykernel import kernelapp as app


In [5]:
X = []
y = []
for feature, label in images_data:
  X.append(feature)
  y.append(label)

In [6]:
X = np.array(X) / 255

In [7]:
X.reshape(-1, img_size, img_size, 1)
y = np.array(y)

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=13)

In [9]:
X_train.shape

(26915, 64, 64, 3)

In [10]:
X_test.shape

(8972, 64, 64, 3)

In [11]:
X_train[0]

array([[[0.65098039, 0.65098039, 0.65098039],
        [0.59215686, 0.59215686, 0.59215686],
        [0.5372549 , 0.5372549 , 0.5372549 ],
        ...,
        [0.18431373, 0.18431373, 0.18431373],
        [0.18039216, 0.18039216, 0.18039216],
        [0.22352941, 0.22352941, 0.22352941]],

       [[0.64313725, 0.64313725, 0.64313725],
        [0.61568627, 0.61568627, 0.61568627],
        [0.54509804, 0.54509804, 0.54509804],
        ...,
        [0.21568627, 0.21568627, 0.21568627],
        [0.18431373, 0.18431373, 0.18431373],
        [0.16862745, 0.16862745, 0.16862745]],

       [[0.64313725, 0.64313725, 0.64313725],
        [0.63137255, 0.63137255, 0.63137255],
        [0.54901961, 0.54901961, 0.54901961],
        ...,
        [0.23921569, 0.23921569, 0.23921569],
        [0.20784314, 0.20784314, 0.20784314],
        [0.17254902, 0.17254902, 0.17254902]],

       ...,

       [[0.49411765, 0.49411765, 0.49411765],
        [0.42352941, 0.42352941, 0.42352941],
        [0.39607843, 0

In [12]:
y_train.shape

(26915,)

In [13]:
y_test.shape

(8972,)

In [14]:
y_train[0]

4

In [15]:
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range = 30,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.2, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip = True,  # randomly flip images
        vertical_flip=False)  # randomly flip images


datagen.fit(X_train)

In [20]:
model = Sequential()
model.add(Conv2D(32,3,padding="same", activation="relu", input_shape=(64,64,3)))
model.add(MaxPool2D())

model.add(Conv2D(32, 3, padding="same", activation="relu"))
model.add(MaxPool2D())

model.add(Conv2D(32, 3, padding="same", activation="elu"))
model.add(MaxPool2D())

model.add(Conv2D(64, 3, padding="same", activation="elu"))
model.add(MaxPool2D())
model.add(Dropout(0.4))

model.add(Flatten())
model.add(Dense(64,activation="relu"))
model.add(Dense(7, activation="softmax"))

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 64, 64, 32)        896       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 16, 16, 32)        9248      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 8, 8, 32)          0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 8, 8, 64)         

In [21]:
model.compile(optimizer = "adam" , loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True) , metrics = ['accuracy'])

In [22]:
early_stop = EarlyStopping(monitor='val_loss', patience=7, verbose=0, mode='min')
mcp_save = ModelCheckpoint('gdrive/My Drive/emotions_data/callback_model.h5', save_best_only=True, monitor='val_loss', mode='min')

In [23]:
history = model.fit(X_train, y_train,
                   validation_data=(X_test, y_test),
                   epochs=40, batch_size=32,
                   verbose=2, callbacks=[early_stop, mcp_save])

Epoch 1/40
842/842 - 142s - loss: 1.6619 - accuracy: 0.3405 - val_loss: 1.5094 - val_accuracy: 0.4075
Epoch 2/40
842/842 - 143s - loss: 1.4494 - accuracy: 0.4412 - val_loss: 1.3742 - val_accuracy: 0.4738
Epoch 3/40
842/842 - 143s - loss: 1.3478 - accuracy: 0.4834 - val_loss: 1.3030 - val_accuracy: 0.5020
Epoch 4/40
842/842 - 144s - loss: 1.2704 - accuracy: 0.5132 - val_loss: 1.2542 - val_accuracy: 0.5224
Epoch 5/40
842/842 - 144s - loss: 1.2127 - accuracy: 0.5356 - val_loss: 1.2144 - val_accuracy: 0.5308
Epoch 6/40
842/842 - 143s - loss: 1.1612 - accuracy: 0.5571 - val_loss: 1.1850 - val_accuracy: 0.5471
Epoch 7/40
842/842 - 143s - loss: 1.1188 - accuracy: 0.5735 - val_loss: 1.1732 - val_accuracy: 0.5543
Epoch 8/40
842/842 - 143s - loss: 1.0732 - accuracy: 0.5952 - val_loss: 1.1848 - val_accuracy: 0.5464
Epoch 9/40
842/842 - 143s - loss: 1.0368 - accuracy: 0.6047 - val_loss: 1.1989 - val_accuracy: 0.5478
Epoch 10/40
842/842 - 142s - loss: 1.0163 - accuracy: 0.6140 - val_loss: 1.1789 - 

In [ ]:
#model.save("gdrive/My Drive/emotions_data/model.h5")